In [5]:
import pandas as pd
import numpy as np
from datetime import date
from dbhelper import PgDB

In [6]:
import holidays

In [28]:
us_holidays = holidays.US(years=range(2007, 2019))
ca_holidays = holidays.CA(years=range(2007, 2019))

In [11]:
query = '''
        insert into publicholiday (date, %s)
        values ('%s', True)
        '''

In [12]:
usmap = {"New Year's Day": 'newyears', 
         "Martin Luther King, Jr. Day": 'us_mlk',
         "Washington's Birthday": 'us_washington',
         "Memorial Day": 'us_memorial',
         "Independence Day": 'us_independence',
         "Labor Day": 'labor',
         "Columbus Day": 'us_columbus',
         "Veterans Day": 'us_veterans',
         "Thanksgiving": 'us_thanksgiving',
         "Christmas Day": 'xmas'
        }

In [27]:
with PgDB() as db:
    for year in range(2017, 2018):
        hdays = holidays.US(years=year)
        for date, name in sorted(hdays.items()):
            if "Observed" not in name:
#                 print name
                db.cur.execute(query % (usmap[name], date))
    
    db.conn.commit()

In [33]:
camap= {"Good Friday": 'ca_goodfriday',
        "Victoria Day": 'ca_victoria',
        "Canada Day": 'ca_canada',
        "Civic Holiday": 'ca_civic',
        "Thanksgiving": 'ca_thanksgiving',
        "Family Day": 'ca_family'
        }

In [35]:
with PgDB() as db:
    for year in range(2017, 2019):
        hdays = holidays.CA(years=year)
        for date, name in sorted(hdays.items()):
            # Ignore if already added as US holiday 
            if date in us_holidays.keys():
                continue
                
            if name not in camap.keys():
                continue
            
            if "Observed" not in name:
#                 print name
                db.cur.execute(query % (camap[name], date))
    
    db.conn.commit()

In [15]:
for date, name in sorted(holidays.CA(years=range(2007, 2017)).items()):
    print date, name

2007-01-01 New Year's Day
2007-04-06 Good Friday
2007-05-21 Victoria Day
2007-07-01 Canada Day
2007-07-02 Canada Day (Observed)
2007-08-06 Civic Holiday
2007-09-03 Labour Day
2007-10-08 Thanksgiving
2007-12-25 Christmas Day
2007-12-26 Boxing Day
2008-01-01 New Year's Day
2008-02-18 Family Day
2008-03-21 Good Friday
2008-05-19 Victoria Day
2008-07-01 Canada Day
2008-08-04 Civic Holiday
2008-09-01 Labour Day
2008-10-13 Thanksgiving
2008-12-25 Christmas Day
2008-12-26 Boxing Day
2009-01-01 New Year's Day
2009-02-16 Family Day
2009-04-10 Good Friday
2009-05-18 Victoria Day
2009-07-01 Canada Day
2009-08-03 Civic Holiday
2009-09-07 Labour Day
2009-10-12 Thanksgiving
2009-12-25 Christmas Day
2009-12-28 Boxing Day (Observed)
2010-01-01 New Year's Day
2010-02-15 Family Day
2010-04-02 Good Friday
2010-05-24 Victoria Day
2010-07-01 Canada Day
2010-08-02 Civic Holiday
2010-09-06 Labour Day
2010-10-11 Thanksgiving
2010-12-24 Christmas Day (Observed)
2010-12-25 Christmas Day
2010-12-27 Boxing Day (O

## Fill in remaining day with all false

In [ ]:
insert into publicholiday 
select distinct(date::timestamp::date) from datefeatures
where date::timestamp::date not in (qselect dates from publicholiday);

## Add mothers day

In [38]:
mday = ['2018-5-13', '2017-5-4', '2016-05-08', '2015-05-10', '2014-5-11', '2013-5-12', '2012-5-13', '2011-5-8', 
        '2010-5-9', '2009-5-10', '2008-5-11', '2007-5-13']
with PgDB() as db:
    for dt in mday:
        db.cur.execute("update publicholiday set mothers=true where date = '%s'" % dt)
    db.conn.commit()

## Add Halloween

In [37]:
with PgDB() as db:
    for year in range(2007, 2019):
        db.cur.execute("update publicholiday set halloween=true where date = '%s-10-31'" % year)
    db.conn.commit()

# Refactor using specialdates table

In [22]:
usmap = {
         "Martin Luther King, Jr. Day": 'mlk',
         "Washington's Birthday": 'presidents',
         "Memorial Day": 'memorial',
         "Independence Day": 'independence',
         "Veterans Day": 'veterans',
         "Thanksgiving": 'thanksgiving'
        }
bothmap = {"Labor Day": 'labor',
           "New Year's Day": 'newyears',
           "Christmas Day": 'xmas'
            }
camap= {"Good Friday": 'goodfriday',
        "Victoria Day": 'victoria',
        "Canada Day": 'canada',
        "Civic Holiday": 'civic',
        "Thanksgiving": 'ca_thanksgiving'
        }

query = '''
        insert into specialdates
        values ('%s', '%s', '%s');
        '''

In [10]:
us_holidays = holidays.US(years=range(2007, 2019))
ca_holidays = holidays.CA(years=range(2007, 2019))

with PgDB() as db:
    for year in range(2007, 2019):
        hdays = holidays.US(years=year)
        for date, name in sorted(hdays.items()):
            if "Observed" not in name:
                if name in usmap:
                    db.cur.execute(query % (date, 'us', usmap[name]))
                elif name in bothmap:
                    db.cur.execute(query % (date, 'zz', bothmap[name]))
    
    db.conn.commit()

In [23]:
with PgDB() as db:
    for year in range(2007, 2019):
        hdays = holidays.CA(years=year)
        for date, name in sorted(hdays.items()):
            if "Observed" not in name:
                if name in camap:
                    db.cur.execute(query % (date, 'ca', camap[name]))
    
    db.conn.commit()

Good Friday 2007-04-06
Victoria Day 2007-05-21
Canada Day 2007-07-01
Civic Holiday 2007-08-06
Thanksgiving 2007-10-08
Good Friday 2008-03-21
Victoria Day 2008-05-19
Canada Day 2008-07-01
Civic Holiday 2008-08-04
Thanksgiving 2008-10-13
Good Friday 2009-04-10
Victoria Day 2009-05-18
Canada Day 2009-07-01
Civic Holiday 2009-08-03
Thanksgiving 2009-10-12
Good Friday 2010-04-02
Victoria Day 2010-05-24
Canada Day 2010-07-01
Civic Holiday 2010-08-02
Thanksgiving 2010-10-11
Good Friday 2011-04-22
Victoria Day 2011-05-23
Canada Day 2011-07-01
Civic Holiday 2011-08-01
Thanksgiving 2011-10-10
Good Friday 2012-04-06
Victoria Day 2012-05-21
Canada Day 2012-07-01
Civic Holiday 2012-08-06
Thanksgiving 2012-10-08
Good Friday 2013-03-29
Victoria Day 2013-05-20
Canada Day 2013-07-01
Civic Holiday 2013-08-05
Thanksgiving 2013-10-14
Good Friday 2014-04-18
Victoria Day 2014-05-19
Canada Day 2014-07-01
Civic Holiday 2014-08-04
Thanksgiving 2014-10-13
Good Friday 2015-04-03
Victoria Day 2015-05-18
Canada Da

### drop Family Day.  Same dates as Presidents day

In [19]:
sorted([k for k,v in ca_holidays.items() if v == 'Family Day'])

[datetime.date(2008, 2, 18),
 datetime.date(2009, 2, 16),
 datetime.date(2010, 2, 15),
 datetime.date(2011, 2, 21),
 datetime.date(2012, 2, 20),
 datetime.date(2013, 2, 18),
 datetime.date(2014, 2, 17),
 datetime.date(2015, 2, 16),
 datetime.date(2016, 2, 15),
 datetime.date(2017, 2, 20),
 datetime.date(2018, 2, 19)]

In [21]:
sorted([k for k,v in us_holidays.items() if v == "Washington's Birthday"])

[datetime.date(2007, 2, 19),
 datetime.date(2008, 2, 18),
 datetime.date(2009, 2, 16),
 datetime.date(2010, 2, 15),
 datetime.date(2011, 2, 21),
 datetime.date(2012, 2, 20),
 datetime.date(2013, 2, 18),
 datetime.date(2014, 2, 17),
 datetime.date(2015, 2, 16),
 datetime.date(2016, 2, 15),
 datetime.date(2017, 2, 20),
 datetime.date(2018, 2, 19)]

## Halloween & Mother's day

In [29]:
with PgDB() as db:
    for year in range(2007, 2019):
        db.cur.execute("insert into specialdates values ('%s-10-31' ,'zz', 'halloween')" % year)
    db.conn.commit()

In [31]:
mday = ['2018-5-13', '2017-5-4', '2016-05-08', '2015-05-10', '2014-5-11', '2013-5-12', '2012-5-13', '2011-5-8', 
        '2010-5-9', '2009-5-10', '2008-5-11', '2007-5-13']
with PgDB() as db:
    for dt in mday:
        db.cur.execute("insert into specialdates values('%s', 'zz', 'mothers')" % dt)
    db.conn.commit()

## Test query for data joining with specialdates with lag/lead

In [41]:
from BorderModel import pd_query

query = '''
        select
            m.date,
            m.metric as waittime,
            s.event,
            s_lead1.event as event_lead1,
            s_lag1.event as event_lag1
        from mungedata m
        left join specialdates s
        on m.date::timestamp::date = s.date
        left join specialdates s_lead1
        on m.date::timestamp::date = s_lead1.date - interval '1 day'
        left join specialdates s_lag1
        on m.date::timestamp::date = s_lag1.date + interval '1 day'
        where
            munger_id = 2
            and crossing_id = 1
            and is_waittime = true
            and m.date >= '1/1/2014'
        '''

df = pd_query(query)

Excluding specific holidays during modeling

```sql
        left join specialdates s
        on m.date::timestamp::date = s.date
        and s.event <> 'mlk'
```

In [42]:
df.head()

,date,waittime,event,event_lead1,event_lag1
0,2014-01-01 00:00:00,0.056481,newyears,None,None
1,2014-01-01 00:05:00,0.056481,newyears,None,None
2,2014-01-01 00:15:00,0.120216,newyears,None,None
3,2014-01-01 00:35:00,0.130700,newyears,None,None
4,2014-01-01 01:15:00,0.879249,newyears,None,None


In [43]:
pd.get_dummies(df.event).head()

,ca_thanksgiving,canada,civic,goodfriday,halloween,independence,labor,memorial,mlk,mothers,newyears,president,thanksgiving,veterans,victoria,xmas
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [37]:
pd.get_dummies(df.event_lead1, prefix='lead1').head()

,lead1_ca_thanksgiving,lead1_canada,lead1_civic,lead1_goodfriday,lead1_halloween,lead1_independence,lead1_labor,lead1_memorial,lead1_mlk,lead1_mothers,lead1_newyears,lead1_president,lead1_thanksgiving,lead1_veterans,lead1_victoria,lead1_xmas
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
[col for col in df.columns.values if 'event' in col]

['event', 'event_lead1', 'event_lag1']